In [1]:
import pandas as pd
import numpy as np
import pickle
import shap
from typing import List, Callable, Optional, Tuple, Any
from sklearn.model_selection import BaseCrossValidator
from sklearn.base import BaseEstimator
from sklearn.utils import resample
from sklearn.linear_model import LinearRegression
from lightgbm import LGBMRegressor
from sklearn.metrics import mean_absolute_percentage_error
from pandas.tseries.offsets import MonthEnd

class DateTimeSeriesSplit:
    def __init__(self, n_splits: int = 4, test_size: int = 1, margin: int = 1, window: int = 3):
        self.n_splits = n_splits
        self.test_size = test_size
        self.margin = margin
        self.window = window
    def get_n_splits(self) -> int:
        return self.n_splits
    def split(self, X: pd.DataFrame, y: Optional[Any] = None, groups: pd.DataFrame = None) -> Tuple[np.ndarray, np.ndarray]:
        unique_dates = sorted(groups.unique())
        rank_dates = {date:rank for rank, date in enumerate(unique_dates)}
        X['index_time'] = groups.map(rank_dates)
        X = X.reset_index(drop=True)
        index_time_list = list(rank_dates.values())
        for i in reversed(range(1, self.n_splits + 1)):
            left_train = int((index_time_list[-1] - i*self.test_size + 1 - self.window - self.margin)*(self.window/np.max([1,self.window])))
            right_train = index_time_list[-1] - i*self.test_size - self.margin + 1
            left_test = index_time_list[-1] - i*self.test_size + 1
            right_test = index_time_list[-1] - (i-1)*self.test_size + 1
            index_test = X.index.get_indexer(X.index[X.index_time.isin(index_time_list[left_test: right_test])])
            index_train = X.index.get_indexer(X.index[X.index_time.isin(index_time_list[left_train: right_train])])
            yield index_train, index_test

class Kraken:
    def __init__(self, estimator: BaseEstimator, cv: BaseCrossValidator, metric: Callable, meta_info_name: str):
        self.estimator = estimator
        self.cv = cv
        self.metric = metric
        self.meta_info_name = meta_info_name
    def get_rank_dict(self, X: np.ndarray, y: np.ndarray, list_of_vars: List[str], group_dt: Optional[np.ndarray]):
        self.dict_fold_importances = {'Feature': list_of_vars, 'abs_shap': np.zeros(len(list_of_vars))}
        for fold, (train_idx, val_idx) in enumerate(self.cv.split(X, groups=group_dt), 1):
            X_train, X_test = X.iloc[train_idx].copy(), X.iloc[val_idx].copy()
            y_train, y_test = y.iloc[train_idx].copy(), y.iloc[val_idx].copy()
            self.estimator.fit(X_train[list_of_vars], y_train.values)
            explainer = shap.Explainer(self.estimator)
            shap_values = explainer.shap_values(X_test[list_of_vars])
            self.dict_fold_importances['abs_shap'] += np.abs(shap_values).mean(axis=0)
        self.fe_dict = {key: value for key, value in zip(self.dict_fold_importances['Feature'], self.dict_fold_importances['abs_shap'])}
        self.rank_dict = {key: rank for rank, key in enumerate(sorted(self.fe_dict, key=self.fe_dict.get, reverse=True), 1)}
    def get_cross_val_score(self, X: np.ndarray, y: np.ndarray, var: str, old_scores: np.ndarray, selected_vars: Optional[List[str]] = None, group_dt: Optional[np.ndarray] = None, round_num: int = 3):
        if selected_vars is None:
            selected_vars = []
        selected_vars.append(var)
        list_scores = []
        for fold, (train_idx, val_idx) in enumerate(self.cv.split(X, groups=group_dt), 1):
            X_train, X_test = X.iloc[train_idx], X.iloc[val_idx]
            y_train, y_test = y.iloc[train_idx], y.iloc[val_idx]
            self.estimator.fit(X_train[selected_vars], y_train)
            error = round(self.metric(np.exp(y_test), np.exp(self.estimator.predict(X_test[selected_vars]))), round_num)
            list_scores.append(error)
        fold_scores = np.array(list_scores)
        summa = sum(fold_scores - old_scores < 0) * 1 + sum(fold_scores - old_scores > 0) * -1
        mean_cv_score = round(np.mean(fold_scores), round_num)
        return fold_scores, summa, mean_cv_score
    def get_vars(self, X: np.ndarray, y: np.ndarray, early_stopping_rounds: int = 30, summa_approve: int = 1, best_mean_cv: int = 100, vars_in_model: Optional[List] = list(), group_dt: Optional[np.ndarray] = None, round_num: int = 3, old_scores: Optional[np.ndarray] = None):
        self.round_num = round_num
        if old_scores is None:
            old_scores = np.array([100 for i in range(self.cv.get_n_splits())])
        iteration_step = 0
        the_list_from_which_we_take_vars = [i for i in list(self.rank_dict.keys()) if i not in vars_in_model]
        feature_was_added = True
        while feature_was_added:
            iteration_step = 0
            var_for_add = ''
            print('начинаем след этап', best_mean_cv)
            best_positive_groups = summa_approve
            for var in the_list_from_which_we_take_vars:
                iteration_step += 1
                if iteration_step > early_stopping_rounds:
                    print(f'early_stopping_rounds {early_stopping_rounds}')
                    break
                fold_scores, summa, mean_cv_score = self.get_cross_val_score(X=X, y=y, var=var, old_scores=old_scores, selected_vars=vars_in_model.copy(), group_dt=group_dt, round_num=self.round_num)
                if (summa > best_positive_groups) or (summa == best_positive_groups and mean_cv_score < best_mean_cv):
                    best_positive_groups = summa
                    best_mean_cv = mean_cv_score
                    old_scores = fold_scores
                    var_for_add = var
                    iteration_step = 0
                    print(f'new var_for_add ! {var_for_add}')
            if var_for_add != '':
                vars_in_model.append(var_for_add)
                the_list_from_which_we_take_vars.remove(var_for_add)
                print('едем дальше')
                print('в итоге получили список', vars_in_model)
                list_meta = ['vars_list'] + [best_positive_groups] + [best_mean_cv] + old_scores.tolist()
                df_meta = pd.DataFrame(list_meta).T
                df_meta.columns = ['vars', 'summa', 'mean_cv_scores'] + ['cv' + str(i) for i in range(1, self.cv.get_n_splits() + 1)]
                df_meta.at[0, 'vars'] = vars_in_model.copy()
                try:
                    df_meta_info = pd.concat([df_meta_info, df_meta])
                except:
                    df_meta_info = df_meta.copy()
                df_meta_info.to_csv(f'df_meta_info_{self.meta_info_name}.csv')
                continue
            else:
                feature_was_added = False
        print('мы сошлись')
        print(vars_in_model)
        print(best_mean_cv)
        return vars_in_model

class AddTrain:
    
    def __init__(self, df_: pd.DataFrame, model_path: str, train_end: str, oot_dates: List[str], vsp_test: np.array, used_features: List[str]):
        self.df_ = df_
        self.model_path = model_path
        self.train_end = train_end
        self.oot_dates = oot_dates
        self.vsp_test = vsp_test
        self.used_features = used_features
    
    def scoring_constant_model(self):
        with open(self.model_path, 'rb') as mod_pkl:
            model = pickle.load(mod_pkl)
        cond1_oot = (self.df_['dt'] > self.train_end)
        X_oot = self.df_[cond1_oot]
        macro_list = []
        print(X_oot['dt'].value_counts().sort_index())
        for dt, subset in X_oot.groupby('dt'):
            y_pred_oot = np.exp(model.predict(subset[self.used_features]))
            mape_oot = round(mean_absolute_percentage_error(np.exp(subset['target']), y_pred_oot), 3)
            macro_oot = round(y_pred_oot.sum(), 2)
            macro_fact = subset['target'].sum()
            ape_macro = round(100*(macro_oot - macro_fact)/macro_fact, 2)
            macro_list.append([dt, mape_oot, macro_oot, macro_fact, ape_macro])
        self.results_scor_constant = pd.DataFrame(macro_list, columns=['dt', 'const_mape_oot', 'const_macro_oot', 'const_macro_fact', 'const_ape_macro'])
    
    def scoring_update_model(self):
        with open(self.model_path, 'rb') as mod_pkl:
            old_model = pickle.load(mod_pkl)
        macro_list = []
        macro_list_valid = []
        for i, _ in enumerate(sorted(self.df_[self.df_['dt'] > self.train_end]['dt'].unique()[0:-3]), 1):
            if i == 1:
                cond1_train = (self.df_['dt'] <= pd.to_datetime(self.train_end) + MonthEnd(len(self.oot_dates) - 1))
                cond2_train = (~self.df_['urf_code_map'].isin(self.vsp_test))
                X_train = self.df_[cond1_train & cond2_train]
                y_train = X_train['target']
                cond1_test = (self.df_['urf_code_map'].isin(self.vsp_test))
                X_test = self.df_[cond1_train & cond1_test]
                y_test = X_test['target']
                cond1_oot = (self.df_['dt'] == pd.to_datetime(self.train_end) + MonthEnd(len(self.oot_dates) + 2))
                X_oot = self.df_[cond1_oot]
                y_oot = X_oot['target']
                cond1_oot_valid = (self.df_['dt'] == pd.to_datetime(self.train_end) + MonthEnd(len(self.oot_dates)))
                X_oot_valid = self.df_[cond1_oot_valid]
                y_oot_valid = X_oot_valid['target']
            elif i > 1:
                cond1_train = (self.df_['dt'] <= pd.to_datetime(self.train_end) + MonthEnd(len(self.oot_dates) + i - 2))
                cond2_train = (~self.df_['urf_code_map'].isin(self.vsp_test))
                X_train = self.df_[cond1_train & cond2_train]
                y_train = X_train['target']
                cond1_test = (self.df_['urf_code_map'].isin(self.vsp_test))
                X_test = self.df_[cond1_train & cond1_test]
                y_test = X_test['target']
                cond1_oot = (self.df_['dt'] == pd.to_datetime(self.train_end) + MonthEnd(len(self.oot_dates) + i + 1))
                X_oot = self.df_[cond1_oot]
                y_oot = X_oot['target']
                cond1_oot_valid = (self.df_['dt'] == pd.to_datetime(self.train_end) + MonthEnd(len(self.oot_dates) + i - 1))
                X_oot_valid = self.df_[cond1_oot_valid]
                y_oot_valid = X_oot_valid['target']
            print('*-*-*-*-*-*-*-*-*-*- start split train/test/oot *-*-*-*-*-*-*-*-*-*-')
            print(f'step_{i}')
            train_test_vc = pd.merge(X_train['dt'].value_counts().sort_index().reset_index(), X_test['dt'].value_counts().sort_index().reset_index(),  how='outer', on='index')
            stats_val_cnt = pd.merge(train_test_vc, X_oot['dt'].value_counts().sort_index().reset_index(), how='outer', on='index')
            final_stats = pd.merge(stats_val_cnt, X_oot_valid['dt'].value_counts().sort_index().reset_index(), how='outer', on='index')
            final_stats.columns = ['dt', 'cnt_train', 'cnt_oos', 'cnt_oot', 'cnt_oot_val']
            final_stats.sort_values(['dt'], axis=0, inplace=True)
            display(final_stats)
            old_params = old_model.get_params()
            old_params['verbosity'] = -1
            model = LGBMRegressor(**old_params)
            model.fit(X_train[self.used_features], y_train)
            dt = X_oot['dt'].unique()[0]
            y_pred_oot = np.exp(model.predict(X_oot[self.used_features]))
            mape_oot = round(mean_absolute_percentage_error(np.exp(X_oot['target']), y_pred_oot), 3)
            macro_oot = round(y_pred_oot.sum(), 2)
            macro_fact = X_oot['target'].sum()
            ape_macro = round(100*(macro_oot - macro_fact)/macro_fact, 2)
            dt_valid = X_oot_valid['dt'].unique()[0]
            y_pred_oot_valid = np.exp(model.predict(X_oot_valid[self.used_features]))
            mape_oot_valid = round(mean_absolute_percentage_error(np.exp(X_oot_valid['target']), y_pred_oot_valid), 3)
            macro_oot_valid = round(y_pred_oot_valid.sum(), 2)
            macro_fact_valid = X_oot_valid['target'].sum()
            ape_macro_valid = round(100*(macro_oot_valid - macro_fact_valid)/macro_fact_valid, 2)
            macro_list.append([dt, mape_oot, macro_oot, macro_fact, ape_macro])
            macro_list_valid.append([dt_valid, mape_oot_valid, macro_oot_valid, macro_fact_valid, ape_macro_valid])
        self.results_scor_update = pd.DataFrame(macro_list, columns=['dt', 'update_w_mape_oot', 'update_w_macro_oot', 'update_w_macro_fact', 'update_w_ape_macro'])
        self.results_scor_update_valid = pd.DataFrame(macro_list_valid, columns=['dt', 'update_w_mape_oot_valid', 'update_w_macro_oot_valid', 'update_w_macro_fact_valid', 'update_w_ape_macro_valid'])
    
    def final_report(self):
        tech_valid_df = pd.merge(self.results_scor_constant, self.results_scor_update_valid, how='inner', on='dt')[['dt', 'const_mape_oot', 'update_w_mape_oot_valid']]
        tech_valid_df['inference_dt'] = pd.to_datetime(tech_valid_df['dt']) + MonthEnd(2)
        report = pd.merge(self.results_scor_constant, self.results_scor_update, how='left', on='dt')
        report['diff_constant'] = round(100*(report['update_w_mape_oot'] - report['const_mape_oot'])/report['const_mape_oot'], 2)
        tech_valid_df2 = pd.merge(tech_valid_df, report[['dt', 'const_mape_oot', 'update_w_mape_oot', 'diff_constant']], how='inner', left_on='inference_dt', right_on='dt').drop('dt_y', axis=1)
        tech_valid_df2['final_quality'] = tech_valid_df2['update_w_mape_oot']
        tech_valid_df2.loc[tech_valid_df2['const_mape_oot_x'] < tech_valid_df2['update_w_mape_oot_valid'], 'final_quality'] = tech_valid_df2['const_mape_oot_y']
        tech_valid_df2['diff_rule'] = round(100*(tech_valid_df2['final_quality'] - tech_valid_df2['const_mape_oot_y'])/tech_valid_df2['const_mape_oot_y'], 2)
        return tech_valid_df2[['dt_x', 'const_mape_oot_x', 'update_w_mape_oot_valid', 'inference_dt', 'const_mape_oot_y', 'update_w_mape_oot', 'final_quality', 'diff_constant', 'diff_rule']]


    def calculate_bootstrap_intervals(model, X_train, y_train, X_oos, y_oos, metric, n_iterations = 1000):
        """
        Функция для расчета 95% доверительных интервалов метрик на train и OOS выборках через бутстрап.
    
        :param model: Обученная модель.
        :param X_train, y_train: Данные для тренировки.
        :param X_oos, y_oos: Данные out-of-sample.
        :param metric: Функция метрики (например, mean_absolute_percentage_error).
        :param n_iterations: Количество итераций бутстрапа.
        :return: 95% доверительные интервалы для метрик на тренировочной и OOS выборках.
        """
        train_metrics = []
        oos_metrics = []
        for i in range(n_iterations):
            # бутстрап выборки
            X_train_resampled, y_train_resampled = resample(X_train, y_train)
            X_oos_resampled, y_oos_resampled = resample(X_oos, y_oos)
            
            # расчет метрик
            train_pred = model.predict(X_train_resampled)
            oos_pred = model.predict(X_oos_resampled)
            
            train_metrics.append(metric(y_train_resampled, train_pred))
            oos_metrics.append(metric(y_oos_resampled, oos_pred))
        
        # расчет 95% доверительных интервалов
        train_lower = np.percentile(train_metrics, 2.5)
        train_upper = np.percentile(train_metrics, 97.5)
        oos_lower = np.percentile(oos_metrics, 2.5)
        oos_upper = np.percentile(oos_metrics, 97.5)
        
        return train_lower, train_upper, oos_lower, oos_upper
    
    
    def scoring_new_model(self, list_of_features: list, start_month: str, window: int, n_splits: int, test_size: int, margin: int, lgbm_params: dict, early_stopping_rounds: int, round_num: int, metric: Callable):
        print(f'Начало выгрузки старой модели. Всего фичей для анализа:{len(list_of_features)}')
        with open(self.model_path, 'rb') as file:
            old_model = pickle.load(file)
        start_month_dt = pd.to_datetime(start_month)
        results = []
        meta_info = []
        macro_list = []
        print(f"Начинаем обработку данных, начиная с {start_month_dt.strftime('%Y-%m')}", end='\n')
        while start_month_dt <= self.df_['dt'].max():
            print('--------------------------------------------------------------------', end='\n')
            print(f"Обрабатываем месяц {start_month_dt.strftime('%Y-%m')}")
            two_month_ago = start_month_dt - pd.DateOffset(months=2)
            two_month_ago_last_day = (two_month_ago + pd.DateOffset(months=1)).replace(day=1) - pd.DateOffset(days=1)
            data_for_model = self.df_[self.df_['dt'] < two_month_ago_last_day].copy()
            train_data = data_for_model[~data_for_model.urf_code_map.isin(self.vsp_test)].copy()
            oos_data = data_for_model[data_for_model.urf_code_map.isin(self.vsp_test)].copy()
            oot_data = self.df_[self.df_['dt'] == two_month_ago_last_day].copy()
            data_for_metric = self.df_[self.df_['dt'] == start_month_dt].copy()
            print(f'Тренировочные данные с {train_data.dt.min()} по {train_data.dt.max()}')
            print(f'Всего  {train_data.shape}')
            print(f'OOT данные с {oot_data.dt.min()} по {oot_data.dt.max()}')
            print(f'Всего  {oot_data.shape}')
            cv_datetime = DateTimeSeriesSplit(window=window, n_splits=n_splits, test_size=test_size, margin=margin)
            group_dt = train_data['dt']
            for fold, (train_idx, val_idx) in enumerate(cv_datetime.split(train_data, groups=group_dt), 1):
                print(fold)
                train, test = train_data.iloc[train_idx], train_data.iloc[val_idx]
                print(f'треин мин {train.dt.min()} треин макс {train.dt.max()} shape {train.shape}')
                print(f'тест мин {test.dt.min()} тест макс {test.dt.max()} shape {test.shape}')
            model = LGBMRegressor(**lgbm_params)
            selector = Kraken(model, cv_datetime, metric, 'updated_model')
            model_linear = LinearRegression()
            selector.get_rank_dict(train_data, train_data['target'], list_of_features, group_dt=group_dt)
            new_vars_class = selector.get_vars(train_data, train_data['target'], vars_in_model=[], early_stopping_rounds=early_stopping_rounds, group_dt=train_data['dt'], round_num=round_num)
            model.fit(train_data[new_vars_class], train_data['target'])
            
            # проверяем на переобучение: 
            
            train_lower, train_upper, oos_lower, oos_upper = calculate_bootstrap_intervals(model, train_data[new_vars_class], train_data['target'], oos_data[new_vars_class], oos_data['target'], metric)
            
            # Сравнение интервалов
            if oos_lower > train_upper or train_lower > oos_upper:
                print("Разница между тренировочными и OOS метриками статистически значима.")
                overfit = 1
            else:
                print("Разница между тренировочными и OOS метриками статистически не значима.")
                
                
            y_pred_new = np.exp(model.predict(oot_data[new_vars_class]))
            mape_new = round(mean_absolute_percentage_error(np.exp(oot_data['target']), y_pred_new), 3)
            y_pred_old = np.exp(old_model.predict(oot_data[self.used_features]))
            mape_old = round(mean_absolute_percentage_error(np.exp(oot_data['target']), y_pred_old), 3)
            date_oot = oot_data['dt'].unique()[0]
            
            if (mape_new < mape_old)&(overfit != 1):
                print(f"Новая модель ({mape_new}) лучше старой ({mape_old}) для {date_oot}")
                old_model = model
                with open(self.model_path, 'wb') as file:
                    pickle.dump(model, file)
                results.append({'month': date_oot, 'model': 'new', 'mape': mape_new})
                self.used_features = [i for i in new_vars_class]
                new_model_flag = 1
            else:
                print(f"Новая модель {mape_new} не лучше старой {mape_old}")
                print("Пробуем обновить старую модель")
                model_new_w = LGBMRegressor(**old_model.get_params())
                model_new_w.fit(train_data[self.used_features], train_data['target'])
                y_pred_update = np.exp(model_new_w.predict(oot_data[self.used_features]))
                mape_update = round(mean_absolute_percentage_error(np.exp(oot_data['target']), y_pred_update), 3)
                if mape_update < mape_old:
                    print(f"старая модель с обновленными весами лучше ({mape_update}) лучше старой ({mape_old}) для {date_oot}")
                    old_model = model_new_w
                    new_model_flag = 0.5
                else:
                    print(f"Старая модель ({mape_old}) лучше новой ({mape_new}) для {date_oot}")
                    results.append({'month': date_oot, 'model': 'old', 'mape': mape_old})
                    new_model_flag = 0
            dt = data_for_metric['dt'].unique()[0]
            y_pred_oot = np.exp(old_model.predict(data_for_metric[self.used_features]))
            y_pred_oos = np.exp(old_model.predict(oos_data[self.used_features]))
            y_pred_train = np.exp(old_model.predict(train_data[self.used_features]))
            mape_oot = round(mean_absolute_percentage_error(np.exp(data_for_metric['target']), y_pred_oot), 3)
            mape_oos = round(mean_absolute_percentage_error(np.exp(oos_data['target']), y_pred_oos), 3)
            mape_train = round(mean_absolute_percentage_error(np.exp(train_data['target']), y_pred_train), 3)
            macro_oot = round(y_pred_oot.sum(), 2)
            macro_fact = np.exp(data_for_metric['target']).sum()
            ape_macro = round(100*(macro_oot - macro_fact)/macro_fact, 2)
            model_linear.fit(train_data[self.used_features].fillna(0), train_data['target'])
            y_pred_oot_lin = np.exp(model_linear.predict(data_for_metric[self.used_features].fillna(0)))
            y_pred_oos_lin = np.exp(model_linear.predict(oos_data[self.used_features].fillna(0)))
            y_pred_train_lin = np.exp(model_linear.predict(train_data[self.used_features].fillna(0)))
            mape_oot_lin = round(mean_absolute_percentage_error(np.exp(data_for_metric['target']), y_pred_oot_lin), 3)
            mape_oos_lin = round(mean_absolute_percentage_error(np.exp(oos_data['target']), y_pred_oos_lin), 3)
            mape_train_lin = round(mean_absolute_percentage_error(np.exp(train_data['target']), y_pred_train_lin), 3)
            macro_oot_lin = round(y_pred_oot_lin.sum(), 2)
            ape_macro_lin = round(100*(macro_oot_lin - macro_fact)/macro_fact, 2)
            macro_list.append([dt, mape_oot, macro_oot, macro_fact, ape_macro, new_model_flag, mape_oos, mape_train, mape_oot_lin, mape_oos_lin, mape_train_lin, macro_oot_lin, ape_macro_lin])
            meta_info.append({
                'month': start_month_dt.strftime('%Y-%m'),
                'features': new_vars_class,
                'mape_new': mape_new,
                'mape_old': mape_old
            })
            next_month = start_month_dt + pd.DateOffset(months=1)
            start_month_dt = (next_month + pd.DateOffset(months=1)).replace(day=1) - pd.DateOffset(days=1)
        meta_info_df = pd.DataFrame(meta_info)
        meta_info_df.to_csv('meta_info.csv', index=False)
        add_stat = pd.DataFrame(macro_list, columns=['dt', 'new_model_mape_oot', 'new_model_macro_oot', 'new_model_macro_fact', 'new_model_ape_macro', 'flag_new_model', 'mape_oos', 'mape_train', 'mape_linear_model', 'mape_oos_lin', 'mape_train_lin', 'macro_lin', 'ape_macro_lin'])
        return pd.DataFrame(results), add_stat


In [2]:
import pandas as pd
import numpy as np
from sklearn.preprocessing import StandardScaler
np.random.seed(0)

# Генерация диапазона дат
dates = pd.date_range(start='2021-01-01', end='2023-12-31', freq='M')

# Идентификаторы магазинов
stores = np.arange(1, 1001)  # Пример для 100 магазинов

# Создание базовой структуры датасета
data = pd.DataFrame({
    'date': np.tile(dates, len(stores)),
    'store_id': np.repeat(stores, len(dates)),
})

# Генерация фичей из нормального распределения и их нормализация
for i in range(1, 23):  # Создаем 23 фичей
    data[f'feature{i}'] = np.random.randn(len(data))

scaler = StandardScaler()
feature_columns = [f'feature{i}' for i in range(1, 23)]
data[feature_columns] = scaler.fit_transform(data[feature_columns])

# Создаем колонку для определения интервала
data['interval'] = pd.cut(data['date'],
                          bins=pd.to_datetime(['2021-01-01', '2021-07-01', '2022-01-01', '2022-07-01', '2024-01-01']),
                          labels=[1, 2, 3, 4])

# Генерация целевой переменной с переменными зависимостями
data['sales'] = 300

# Первый интервал
data.loc[data['interval'] == 1, 'sales'] += (data['feature1']**2 + data['feature2']**2) * 50

# Второй интервал
data.loc[data['interval'] == 2, 'sales'] += (data['feature1']**2 + data['feature2']**2 + data['feature3']) * 50

# Третий интервал
data.loc[data['interval'] == 3, 'sales'] += (data['feature4']**2 + data['feature1']**2 + data['feature3'] + data['feature5']) * 50

# Четвертый интервал
data.loc[data['interval'] == 4, 'sales'] += (data['feature1']+ data['feature5']**2 + data['feature2']**2 + data['feature3'] + data['feature6']) * 50

# Добавление положительного шума
data['sales'] += np.abs(np.random.randn(len(data))) * 20

# Преобразование даты в последний день месяца
data['date'] = data['date'] + pd.offsets.MonthEnd(0)

data.head()


,date,store_id,feature1,feature2,feature3,feature4,feature5,feature6,feature7,feature8,...,feature15,feature16,feature17,feature18,feature19,feature20,feature21,feature22,interval,sales
0,2021-01-31,1,1.775986,-0.390906,0.149609,-1.018778,-1.247573,0.610023,-1.081291,-0.648393,...,-0.860254,-1.608853,-0.199029,-1.554852,-1.472078,-1.330927,-0.781315,0.181685,1,491.673674
1,2021-02-28,1,0.406903,0.383177,-1.983944,0.841073,-0.214757,-0.374094,0.389578,-0.345397,...,0.333635,-1.462329,-1.076121,-1.410686,-0.148482,-0.699175,0.069158,-0.925021,1,327.640335
2,2021-03-31,1,0.987685,-0.215193,-0.051645,0.802589,-0.375916,0.249239,-1.024414,0.812455,...,-2.206935,-0.419288,0.480529,-0.982141,-1.089411,0.537801,-1.180819,-0.256047,1,372.621269
3,2021-04-30,1,2.254641,-1.282657,1.885196,-0.408023,-0.626370,-0.735151,0.405136,0.521513,...,0.087990,0.044734,0.117774,-0.261722,0.232714,0.529142,0.006985,0.902975,1,641.431389
4,2021-05-31,1,1.879886,-0.555043,-0.752235,2.254661,-1.399519,0.509127,0.053544,0.398179,...,0.634113,-0.529545,-0.129623,1.570673,0.358058,-0.522310,2.256250,1.419941,1,538.631413


In [3]:
data['dt'] = data.date
data['urf_code_map'] = data.store_id
data['target'] = np.log(data.sales)

In [4]:
# список фичей для включения в модель
model_list = ['feature' + str(i) for i in range(1,5)]; model_list

['feature1', 'feature2', 'feature3', 'feature4']

In [5]:
from sklearn.metrics import mean_absolute_percentage_error as mape
# создать базовую модель

data_sample = data[data['dt'] <'2022-07-31']
data_sample.shape

(18000, 29)

In [6]:
model_base = LGBMRegressor(n_jobs=-1)

In [7]:
model_base.fit(data_sample[model_list], data_sample['target'])

LGBMRegressor()

In [8]:
mape(np.exp(data_sample['target']), np.exp(model_base.predict(data_sample[model_list])))

0.09832746939337923

In [9]:
data_2023 = data[data['dt'] >'2022-12-31']

In [10]:
mape(np.exp(data_2023['target']), np.exp(model_base.predict(data_2023[model_list])))

0.263833417490779

In [11]:
with open ("model_base.pkl", 'wb') as model_alpha:
    pickle.dump(model_base, model_alpha)

In [12]:
list_stores = data.urf_code_map.unique().tolist()

In [13]:
import random
random.seed(0)
subset_size = 500
oos_list = random.sample(list_stores, subset_size)

In [14]:
new_exp = AddTrain(df_ = data, 
                   model_path='model_base.pkl', 
                   train_end = '2022-10-31', 
                   oot_dates= ['2022-11-30', '2022-12-31'],
                   vsp_test = oos_list,
                   used_features = model_list)

In [15]:
new_exp.scoring_constant_model()

2022-11-30    1000
2022-12-31    1000
2023-01-31    1000
2023-02-28    1000
2023-03-31    1000
2023-04-30    1000
2023-05-31    1000
2023-06-30    1000
2023-07-31    1000
2023-08-31    1000
2023-09-30    1000
2023-10-31    1000
2023-11-30    1000
2023-12-31    1000
Name: dt, dtype: int64


In [16]:
new_exp.scoring_update_model()

*-*-*-*-*-*-*-*-*-*- start split train/test/oot *-*-*-*-*-*-*-*-*-*-
step_1


Passing 'suffixes' which cause duplicate columns {'dt_x'} in the result is deprecated and will raise a MergeError in a future version.


,dt,cnt_train,cnt_oos,cnt_oot,cnt_oot_val
0,2021-01-31,500.0,500.0,NaN,NaN
1,2021-02-28,500.0,500.0,NaN,NaN
2,2021-03-31,500.0,500.0,NaN,NaN
3,2021-04-30,500.0,500.0,NaN,NaN
4,2021-05-31,500.0,500.0,NaN,NaN
5,2021-06-30,500.0,500.0,NaN,NaN
6,2021-07-31,500.0,500.0,NaN,NaN
7,2021-08-31,500.0,500.0,NaN,NaN
8,2021-09-30,500.0,500.0,NaN,NaN
9,2021-10-31,500.0,500.0,NaN,NaN


*-*-*-*-*-*-*-*-*-*- start split train/test/oot *-*-*-*-*-*-*-*-*-*-
step_2


Passing 'suffixes' which cause duplicate columns {'dt_x'} in the result is deprecated and will raise a MergeError in a future version.


...

Passing 'suffixes' which cause duplicate columns {'dt_x'} in the result is deprecated and will raise a MergeError in a future version.


,dt,cnt_train,cnt_oos,cnt_oot,cnt_oot_val
0,2021-01-31,500.0,500.0,NaN,NaN
1,2021-02-28,500.0,500.0,NaN,NaN
2,2021-03-31,500.0,500.0,NaN,NaN
3,2021-04-30,500.0,500.0,NaN,NaN
4,2021-05-31,500.0,500.0,NaN,NaN
5,2021-06-30,500.0,500.0,NaN,NaN
6,2021-07-31,500.0,500.0,NaN,NaN
7,2021-08-31,500.0,500.0,NaN,NaN
8,2021-09-30,500.0,500.0,NaN,NaN
9,2021-10-31,500.0,500.0,NaN,NaN


*-*-*-*-*-*-*-*-*-*- start split train/test/oot *-*-*-*-*-*-*-*-*-*-
step_11


Passing 'suffixes' which cause duplicate columns {'dt_x'} in the result is deprecated and will raise a MergeError in a future version.


,dt,cnt_train,cnt_oos,cnt_oot,cnt_oot_val
0,2021-01-31,500.0,500.0,NaN,NaN
1,2021-02-28,500.0,500.0,NaN,NaN
2,2021-03-31,500.0,500.0,NaN,NaN
3,2021-04-30,500.0,500.0,NaN,NaN
4,2021-05-31,500.0,500.0,NaN,NaN
5,2021-06-30,500.0,500.0,NaN,NaN
6,2021-07-31,500.0,500.0,NaN,NaN
7,2021-08-31,500.0,500.0,NaN,NaN
8,2021-09-30,500.0,500.0,NaN,NaN
9,2021-10-31,500.0,500.0,NaN,NaN


In [17]:
report = new_exp.final_report()

In [20]:
params = {'max_depth': 3,
 'n_estimators': 100,
 'objective': 'mape',
 'boosting_type': 'goss',
 'verbose': -1,
 'n_jobs': -1}

In [21]:
import sys

# Открытие файла логов для записи
with open('log_file_scoring_new_model.txt', 'w') as f:
    # Сохранение оригинального стандартного вывода
    original_stdout = sys.stdout
    try:
        # Перенаправление стандартного вывода в файл
        sys.stdout = f
        
        # Теперь все вызовы print будут записываться в файл 'log_file.txt'
        rez, stat = new_exp.scoring_new_model(start_month = '2023-01-31',list_of_features= [f'feature{i}' for i in range(1,23)], 
                                              window = 18, n_splits = 3, test_size = 1, margin = 0, 
                          lgbm_params= params, early_stopping_rounds= 5, round_num= 3, metric= mape)
        # Здесь можно разместить остальной код, вывод которого должен быть сохранён в файле
    finally:
        # Возвращение стандартного вывода в исходное состояние
        sys.stdout = original_stdout

In [22]:
rez

,month,model,mape
0,2022-11-30,new,0.212
1,2022-12-31,new,0.192
2,2023-01-31,new,0.184
3,2023-02-28,new,0.192
4,2023-03-31,new,0.165
5,2023-04-30,new,0.174
6,2023-05-31,new,0.160
7,2023-06-30,new,0.160
8,2023-07-31,new,0.155
9,2023-09-30,new,0.142


In [ ]:
stat

,dt,new_model_mape_oot,new_model_macro_oot,new_model_macro_fact,new_model_ape_macro,flag_new_model,mape_oos,mape_train,mape_linear_model,mape_oos_lin,mape_train_lin,macro_lin,ape_macro_lin
0,2023-01-31,0.203,350979.76,420327.161881,-16.50,1.0,0.182,0.209,0.226,0.192,0.230,403463.85,-4.01
1,2023-02-28,0.220,352936.83,414969.158558,-14.95,1.0,0.183,0.195,0.257,0.194,0.229,404893.89,-2.43
2,2023-03-31,0.185,365315.18,412780.214521,-11.50,1.0,0.156,0.175,0.227,0.191,0.222,401852.41,-2.65
3,2023-04-30,0.186,365326.21,413833.505866,-11.72,1.0,0.157,0.172,0.236,0.192,0.221,403834.99,-2.42
4,2023-05-31,0.167,366946.62,424145.684191,-13.49,1.0,0.157,0.174,0.201,0.193,0.221,407951.11,-3.82
5,2023-06-30,0.170,364828.85,416622.993197,-12.43,1.0,0.160,0.177,0.214,0.193,0.220,404116.86,-3.00
6,2023-07-31,0.165,362571.88,414154.856952,-12.45,1.0,0.158,0.173,0.212,0.193,0.220,402399.71,-2.84
7,2023-08-31,0.156,364913.08,417887.429617,-12.68,1.0,0.158,0.172,0.201,0.193,0.219,404371.62,-3.23
8,2023-09-30,0.147,364627.46,414133.204456,-11.95,1.0,0.157,0.164,0.195,0.194,0.218,405674.48,-2.04
9,2023-10-31,0.153,364322.41,417891.380337,-12.82,0.5,0.156,0.167,0.205,0.194,0.217,404626.22,-3.17


In [ ]:
# Соединяем статистики всех экспериментов для сравнения
pd.merge(report, stat, on = 'dt')[['dt', 'const_mape_oot', 'update_w_mape_oot', 'new_model_mape_oot', 'mape_linear_model']]

,dt,const_mape_oot,update_w_mape_oot,new_model_mape_oot,mape_linear_model
0,2023-01-31,0.255,NaN,0.203,0.226
1,2023-02-28,0.289,0.244,0.220,0.257
2,2023-03-31,0.265,0.221,0.185,0.227
3,2023-04-30,0.305,0.248,0.186,0.236
4,2023-05-31,0.241,0.196,0.167,0.201
5,2023-06-30,0.261,0.204,0.170,0.214
6,2023-07-31,0.277,0.214,0.165,0.212
7,2023-08-31,0.241,0.193,0.156,0.201
8,2023-09-30,0.253,0.185,0.147,0.195
9,2023-10-31,0.261,0.202,0.153,0.205


In [ ]:
dd = pd.read_csv('dd.csv')

In [ ]:
dd[['dt', 'const_mape_oot', 'update_w_mape_oot', 'new_model_mape_oot']]

,dt,const_mape_oot,update_w_mape_oot,new_model_mape_oot
0,2023-01-31,0.255,NaN,0.203
1,2023-02-28,0.289,0.244,0.220
2,2023-03-31,0.265,0.221,0.185
3,2023-04-30,0.305,0.248,0.186
4,2023-05-31,0.241,0.196,0.167
5,2023-06-30,0.261,0.204,0.170
6,2023-07-31,0.277,0.214,0.165
7,2023-08-31,0.241,0.193,0.156
8,2023-09-30,0.253,0.185,0.147
9,2023-10-31,0.261,0.202,0.153
